![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Multimodal - Audio
</b></font> </br></p>


---

In [1]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()
# Bei Bedarf: Trennen zwischen Installationsname () und Importname (für Python) beide Angaben in Klammern
# install_packages([('markitdown[all]', 'markitdown'), 'langchain_chroma', ]

✓ OPENAI_API_KEY erfolgreich gesetzt
✓ HF_TOKEN erfolgreich gesetzt

Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

Installierte LangChain-Bibliotheken:
langchain                                1.0.3
langchain-chroma                         1.0.0
langchain-classic                        1.0.0
langchain-community                      0.4.1
langchain-core                           1.0.3
langchain-ollama                         1.0.0
langchain-openai                         1.0.2
langchain-text-splitters                 1.0.0

Installierte LangGraph-Bibliotheken:
langgraph                                1.0.2
langgraph-checkpoint                     3.0.0
langgraph-prebuilt                       1.0.2
langgraph-sdk                            0.2.9

IP-Adresse: 35.230.165.41
Hostname: 41.165.230.35.bc.googleusercontent.com
Stadt: Washington
Region: District of Columbia
Land: US
Koordinaten: 38.8951,-77.0364
Provider: AS396982 Google LLC
Postleitzahl: 20004
Zeitzone: Am

In [34]:
#@title 📂 Audio { display-mode: "form" }
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02_daten/03_audio/AI_vs_Human_Thinking.mp3 -o AI_vs_Human_Thinking.mp3
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02_daten/03_audio/elevenlabs_erlkoenig_valentino.mp3 -o elevenlabs_erlkoenig_valentino.mp3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0    104      0 --:--:-- --:--:-- --:--:--   105


In [2]:
#@title 🛠️ Walkaround Gradio/Colab - 2 x starten  { display-mode: "form" }
#
# Walkaround Gradio/Colab nach der Ausführung funktioniert auch Gradio Mikro 😊😊
# Code erstellt keine Audio-Aufnahme
#
from IPython.display import Javascript, display, HTML
from google.colab import output
from base64 import b64decode
import io
from IPython.display import Audio
from pydub import AudioSegment
import time

# Verbessertes JavaScript mit besserer Fehlerbehandlung
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));

const b2text = blob => new Promise(resolve => {
    const reader = new FileReader();
    reader.onloadend = e => resolve(e.srcElement.result);
    reader.readAsDataURL(blob);
});

var record = async function(time) {
    try {
        // Explizite Anfrage für Medienzugriff mit Fehlermeldung
        const stream = await navigator.mediaDevices.getUserMedia({ audio: true })
            .catch(err => {
                console.error("Mikrofonzugriff verweigert:", err.message);
                return null;
            });

        if (!stream) {
            return "ERROR: Mikrofonzugriff nicht möglich";
        }

        // Recorder erstellen und konfigurieren
        const recorder = new MediaRecorder(stream);
        const chunks = [];

        recorder.ondataavailable = e => chunks.push(e.data);

        // Promise für das Ende der Aufnahme
        const recordingPromise = new Promise(resolve => {
            recorder.onstop = async () => {
                const blob = new Blob(chunks);
                const text = await b2text(blob);

                // Alle Tracks beenden um Ressourcen freizugeben
                stream.getTracks().forEach(track => track.stop());

                resolve(text);
            };
        });

        // Aufnahme starten
        recorder.start();
        console.log("Aufnahme gestartet...");

        // Warten und dann stoppen
        await sleep(time);
        recorder.stop();
        console.log("Aufnahme beendet.");

        // Auf das Ergebnis warten
        return await recordingPromise;
    } catch (err) {
        console.error("Fehler bei der Aufnahme:", err.message);
        return "ERROR: " + err.message;
    }
}
"""

def record(seconds=3):
    # Benutzer informieren
    print(f"Bereite Aufnahme vor ({seconds} Sekunden)...")
    print("Bitte erteilen Sie die Mikrofonzugriffsberechtigung, wenn der Browser danach fragt.")

    # JavaScript-Code ausführen
    display(Javascript(RECORD))

    # Kurze Pause, um sicherzustellen, dass JavaScript geladen wurde
    time.sleep(1)

    # Explizite Benutzer-Anweisung
    display(HTML("""
    <div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; margin: 10px 0;">
        <p><b>Hinweis:</b> Wenn ein Dialog zur Mikrofonberechtigung erscheint, wählen Sie bitte "Zulassen".</p>
    </div>
    """))

    # Aufnahme starten mit Timeout-Handhabung
    try:
        print(f"Aufnahme läuft für {seconds} Sekunden. Bitte sprechen Sie jetzt...")

        # Timeout-Wert etwas höher als die eigentliche Aufnahmezeit setzen
        result = output.eval_js(f'record({seconds * 1000})', timeout_sec=(seconds + 5))

        # Prüfen auf Fehler
        if isinstance(result, str) and result.startswith("ERROR"):
            print(f"Fehler aufgetreten: {result}")
            return None

        # Audio dekodieren
        binary = b64decode(result.split(',')[1])

        # In AudioSegment konvertieren
        audio = AudioSegment.from_file(io.BytesIO(binary), format="webm")
        print("Aufnahme erfolgreich abgeschlossen!")

        return audio

    except Exception as e:
        print(f"Fehler bei der Aufnahme: {str(e)}")
        print("Tipp: Stellen Sie sicher, dass Sie die Mikrofonberechtigung erteilt haben.")
        return None

# Aufnahme starten
# print("Drücken Sie Enter, um die Aufnahme zu starten...")
# input()
audio = record(1)
audio = record(1)

# Wenn die Aufnahme erfolgreich war, abspielen
# if audio is not None:
#     display(Audio(data=audio.export(format="wav").read(), rate=audio.frame_rate))
#     print("Sie können die Audio-Datei oben abspielen.")

Bereite Aufnahme vor (1 Sekunden)...
Bitte erteilen Sie die Mikrofonzugriffsberechtigung, wenn der Browser danach fragt.


<IPython.core.display.Javascript object>

Aufnahme läuft für 1 Sekunden. Bitte sprechen Sie jetzt...
Fehler aufgetreten: ERROR: Mikrofonzugriff nicht möglich
Bereite Aufnahme vor (1 Sekunden)...
Bitte erteilen Sie die Mikrofonzugriffsberechtigung, wenn der Browser danach fragt.


<IPython.core.display.Javascript object>

Aufnahme läuft für 1 Sekunden. Bitte sprechen Sie jetzt...
Fehler aufgetreten: ERROR: Mikrofonzugriff nicht möglich


# 1 | Einführung
---

Künstliche Intelligenz eröffnet vielfältige Möglichkeiten zur Analyse, Erzeugung und Weiterverarbeitung von Audiodaten. Die folgende Übersicht gliedert zentrale Aufgabenbereiche der KI-gestützten Audiotechnologie in thematische Kategorien. Sie reicht von der Klassifikation und Sprachanalyse über die synthetische Sprach- und Musikgenerierung bis hin zu Sicherheits- und Strukturierungsmaßnahmen – und bietet so einen praxisnahen Einblick in die aktuellen Einsatzfelder intelligenter Audiotechnik.

| **Kategorie**                        | **Aufgabe**                               | **Beschreibung**                                                                              |
| ------------------------------------ | ----------------------------------------- | --------------------------------------------------------------------------------------------- |
| 🧠 **Analyse & Klassifikation**      | **Audio-Klassifikation**                  | Zuordnung von Audioinhalten zu Kategorien (z. B. Musik, Sprache, Geräusch).                   |
|                                      | Sprechererkennung                         | Identifikation oder Verifikation einer Person anhand ihrer Stimme.                            |
|                                      | **Emotionserkennung**                         | Analyse der Stimme zur Einschätzung von Emotionen (z. B. wütend, fröhlich).                   |
|                                      | **Spracherkennung**                           | Erkennung der gesprochenen Sprache (z. B. Deutsch, Englisch, Französisch).                    |
|                                      | Ereigniserkennung                         | Identifikation spezifischer Geräusche oder akustischer Ereignisse (z. B. Sirene, Türknallen). |
|                                      | Musikgenre-Klassifikation                 | Automatische Erkennung des Musikstils (z. B. Jazz, Rock, Klassik).                            |
| 📝 **Generierung & Transformation**  | **Text-to-Speech (TTS)**                  | Umwandlung von Text in natürliche Sprache (synthetische Sprachausgabe).                       |
|                                      | Voice Cloning                             | Imitation einer bestimmten Stimme für synthetische Sprachgenerierung.                         |
|                                      | Sprachstiltransfer                        | Veränderung von Tonlage, Akzent oder Sprechstil (z. B. neutral ↔ emotional).                  |
|                                      | Musikgenerierung                          | Automatisches Erzeugen von Musikstücken oder Melodien durch KI.                               |
|                                      | Audio-Super-Resolution                    | Verbesserung der Klangqualität (z. B. Hochskalierung der Abtastrate).                         |
|                                      | Audio-Restauration                        | Entfernung von Rauschen, Klicks oder Verzerrungen aus alten Aufnahmen.                        |
| 🧩 **Ergänzung & Vervollständigung** | Lückenfüllung in Audio                    | Ergänzung fehlender Audiosegmente (z. B. durch KI erzeugte Übergänge).                        |
|                                      | Audio-Inpainting                          | Rekonstruktion beschädigter oder unvollständiger Audiostücke.                                 |
|                                      | Sprachfortsetzung                     | Fortsetzung einer Rede oder Geschichte auf Basis eines gesprochenen Anfangs.                  |
| 📚 **Informationsgewinnung**         | **Spracherkennung (ASR)**                 | Umwandlung gesprochener Sprache in geschriebenen Text.                                        |
|                                      | Schlüsselworterkennung (Keyword Spotting) | Detektion bestimmter Wörter oder Befehle im Audio (z. B. „Hey Siri“).                         |
|                                      | **Audio-Transkription**                      | Erstellung eines schriftlichen Transkripts (z. B. für Podcasts, Meetings).                    |
|                                      | Audio-Indexierung & -Suche                | Durchsuchbarkeit großer Audiodatenbanken durch akustische Merkmale.                           |
| 🛡️ **Sicherheit & Strukturierung**  | Deepfake-Stimmerkennung                   | Erkennung synthetisch erzeugter oder gefälschter Stimmen.                                     |
|                                      | Anonymisierung                            | Veränderung der Stimme zur Wahrung der Privatsphäre (z. B. Verzerrung).                       |
|                                      | Audio-Wasserzeichen                       | Einbettung oder Erkennung unsichtbarer Marker zum Schutz vor Missbrauch.                      |
|                                      | Formatkonvertierung                       | Umwandlung von Audioformaten (z. B. WAV ↔ MP3).                                               |


# 2 | Speech-to-Text (STT)
---

Die Umwandlung gesprochener Sprache in Text, bekannt als Audio-Transkription, ermöglicht die automatische Verschriftlichung von Gesprächen, Interviews oder Vorträgen. Dies erleichtert die Dokumentation, Archivierung und Weiterverarbeitung von Audioinhalten. Moderne Spracherkennungstechnologien nutzen KI-Modelle, um Sprache präzise zu erfassen und in lesbaren Text umzuwandeln.  

Transkriptionen sind in vielen Bereichen von Bedeutung – von der journalistischen Berichterstattung über die Erstellung von Untertiteln bis hin zur Analyse von Kundenfeedback. Durch den Einsatz automatisierter Systeme kann dieser Prozess erheblich beschleunigt werden, wodurch große Mengen an Audio effizient verarbeitet werden können.

In [27]:
import openai
from google.colab import files
from IPython.display import display, Markdown

# 📂 Nutzer lädt eine Audiodatei hoch
# uploaded = files.upload()
# audio_file_path = list(uploaded.keys())[0]

# Die Audiodatei wird direkt verwendet, anstatt sie hochzuladen
audio_file_path = "elevenlabs_erlkoenig_valentino.mp3"

# Audiodatei öffnen und Whisper nutzen
with open(audio_file_path, "rb") as audio_file:
    response = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
    )

# Markdown-Formatierte Ausgabe
markdown_output = f"""
# 🎙️ Transkript der Audiodatei

**📂 Datei:** `{audio_file_path}`
**🤖 Modell:** `Whisper-1`

---

{response.text}
"""

# Markdown in Google Colab anzeigen
display(Markdown(markdown_output))


# 🎙️ Transkript der Audiodatei

**📂 Datei:** `elevenlabs_erlkoenig_valentino.mp3`
**🤖 Modell:** `Whisper-1`

---

Wer reitet so spät durch Nacht und Wind? Es ist der Vater mit seinem Kind. Er hat den Knaben wohl in dem Arm, Er fasst ihn sicher, er hält ihn warm. Mein Sohn, was birgst du so bang dein Gesicht? Siehst, Vater, du den Erlkönig nicht? Den Erlenkönig mit Kron und Schweif? Mein Sohn, es ist ein Nebelstreif. Du liebes Kind, komm, geh mit mir, Gar schöne Spiele spiel ich mit dir, Manch bunte Blumen sind an dem Strand, Meine Mutter hat manch Güldengewand. Mein Vater, mein Vater, und hörest du nicht, Was Erlenkönig mir leise verspricht? Sei ruhig, bleibe ruhig, mein Kind, In dürren Blättern säuselt der Wind. Willst, feiner Knabe, du mit mir gehen? Meine Töchter sollen dich warten schön, Meine Töchter führen den nächtlichen Reihen Und wiegen und tanzen und singen dich ein. Mein Vater, mein Vater, und siehst du nicht dort Erlkönigs Töchter am düstern Ort? Mein Sohn, mein Sohn, ich seh es genau, Es scheinen die alten Weiden so grau. Ich liebe dich, mich reizt deine schöne Gestalt, Und bist du nicht willig, so brauch ich Gewalt. Mein Vater, mein Vater, jetzt fasst er mich an, Erlkönig hat mir ein Leids getan. Dem Vater grausets, er reitet geschwind, Er hält in Armen das ächzende Kind, Er reicht den Hof mit Mühe und Not, In seinen Armen das Kind war tot.


<p><font color='black' size="5">
Transkript mit Zeitstempel
</font></p>

In [4]:
with open(audio_file_path, "rb") as audio_file:
    response = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        response_format="verbose_json"
    )

# Zugriff auf die Segment-Zeitstempel (Sätze)
display(Markdown("## 📄 Transkript mit Zeitstempel"))
display(Markdown(f"---"))

for segment in response.segments:
    display(Markdown(f"{segment.start:.2f}s - {segment.end:.2f}s: {segment.text}"))

## 📄 Transkript mit Zeitstempel

---

0.00s - 2.52s:  Wer reitet so spät durch Nacht und Wind?

2.52s - 4.60s:  Es ist der Vater mit seinem Kind.

4.60s - 7.00s:  Er hat den Knaben wohl in dem Arm,

...

72.32s - 74.80s:  Erlkönig hat mir ein Leids getan.

74.80s - 77.88s:  Dem Vater grausets, er reitet geschwind,

77.88s - 80.52s:  Er hält in Armen das ächzende Kind,

80.52s - 83.08s:  Er reicht den Hof mit Mühe und Not,

83.08s - 85.08s:  In seinen Armen das Kind war tot.

# 3 | Text-to-Speech (TTS)
---

Fokus ist die Anwendung von Text-to-Speech (TTS) Large Language Models (LLMs) mit den fortschrittlichen Lösungen von OpenAI. Im Mittelpunkt steht das Modell TTS-1, ein leistungsfähiges und vielseitiges System zur Umwandlung von geschriebenem Text in natürliche Sprache. TTS-1 wurde für den Echtzeiteinsatz optimiert und eignet sich daher besonders für Anwendungen, die eine geringe Latenz erfordern. Das Modell repräsentiert einen bedeutenden Fortschritt in der Sprachsynthese, da es mithilfe von Deep-Learning-Techniken qualitativ hochwertige und realistisch klingende Sprachausgaben erzeugt. Im Rahmen dieses Moduls werden die Funktionalitäten von TTS-1 untersucht, seine praktischen Einsatzmöglichkeiten analysiert und sein Einfluss auf verschiedene Branchen betrachtet – von barrierefreien Kommunikationslösungen bis hin zu interaktiven Sprachantwortsystemen.

<p><font color='black' size="5">
Stimmen
</font></p>

Dieser Codeausschnitt demonstriert die Nutzung der Text-to-Speech-API von OpenAI zur Erzeugung gesprochener Sprache aus Text. Zunächst werden die benötigten Bibliotheken importiert: **openai** für die API-Kommunikation, **IPython.display** zur Audiowiedergabe in Jupyter-Notebooks und **base64** für die Kodierung. Die Variable `TEXT` enthält den zu konvertierenden Text. Anschließend wird die Funktion `openai.audio.speech.create()` mit drei Parametern aufgerufen: dem Modell (`"tts-1"`), der gewählten Stimme (`"alloy"`) und dem Eingabetext.  

OpenAI stellt mehrere Stimmen mit unterschiedlichen Charakteristiken zur Verfügung:  

* **Alloy** – neutral  
* **Echo** – jugendlich  
* **Fable** – männlich  
* **Onyx** – tiefe männliche Stimme  
* **Nova** – weiblich  
* **Shimmer** – warme weibliche Stimme  

Jede dieser Stimmen hat eigene klangliche Eigenschaften, sodass je nach Anwendung die passende ausgewählt werden kann. Zudem bietet OpenAI mit `"tts-1-hd"` eine hochauflösende Variante für eine verbesserte Audioqualität, die jedoch mit einer höheren Latenz verbunden sein kann. Die Funktion liefert ein Antwortobjekt zurück, aus dem der generierte Audioinhalt extrahiert und in der Variable `audio_data` zur weiteren Verarbeitung oder Wiedergabe gespeichert wird.

In [5]:
import openai
from IPython.display import display, Audio

# 📝 Text
summary_text = """
In dem Gedicht reitet ein Vater mit seinem Sohn durch die Nacht, während der Junge von dem Erlkönig träumt, der ihn mit verlockenden Angeboten anlockt. Der Vater versucht, seinen Sohn zu beruhigen und erklärt, dass die Erscheinungen nur Nebel sind. Doch der Erlkönig wird bedrohlich und fordert den Jungen, was schließlich zu einem tragischen Ende führt, als der Vater mit seinem tot geglaubten Kind nach Hause zurückkehrt.
"""

# 🎤 Verfügbare OpenAI-Stimmen
voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]

# 📂 Audiodateien für jede Stimme generieren und speichern
audio_files = []
for voice in voices:
    tts_response = openai.audio.speech.create(
        model="tts-1",
        voice=voice,
        input=summary_text
    )

    # 📂 Speichern der Audiodatei
    audio_filename = f"summary_{voice}.mp3"
    with open(audio_filename, "wb") as audio_file:
        audio_file.write(tts_response.content)

    audio_files.append(audio_filename)

In [6]:
# Ausgabe
display(Markdown("## 🔊 ... Speech"))
display(Markdown("---"))

# 🔊 Ausgabe der Audiodateien
for file in audio_files:
    print(f"🎧 {file} erstellt und abspielbar!")
    display(Audio(file, autoplay=False))
    print()

Output hidden; open in https://colab.research.google.com to view.

# 4 | Audio-Summary
---

Die automatische Audio-Zusammenfassung ermöglicht es, lange Sprachaufnahmen auf ihre wesentlichen Inhalte zu reduzieren. Mithilfe von KI-gestützten Algorithmen werden zentrale Aussagen erkannt und in kompakter Form wiedergegeben. Dies erleichtert die schnelle Erfassung wichtiger Informationen, ohne das gesamte Audio anhören zu müssen.  

Diese Technik wird häufig für Podcasts, Meetings oder Interviews eingesetzt, um relevante Inhalte effizient zusammenzufassen. Durch den Einsatz moderner Spracherkennung und natürlicher Sprachverarbeitung (NLP) können präzise und verständliche Zusammenfassungen erstellt werden.

In [28]:
import openai
from google.colab import files
from IPython.display import display, Markdown

# 📂 Nutzer lädt eine Audiodatei hoch
# uploaded = files.upload()
# audio_file_path = list(uploaded.keys())[0]

# Die Audiodatei wird direkt verwendet, anstatt sie hochzuladen
audio_file_path = "AI_vs_Human_Thinking.mp3"

# 🎙️ Audiodatei transkribieren mit Whisper
with open(audio_file_path, "rb") as audio_file:
    response = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )

# 📝 Transkribierter Text
transcribed_text = response.text

# 🧠 GPT-4o-mini: Zusammenfassung
gpt_prompt = f"""
Erstelle eine kurze Zusammenfassung des folgenden Textes. Die Zusammenfassung sollte maximal 3-4 Sätze umfassen und die Hauptpunkte klar darstellen.

### Transkript:
{transcribed_text}
"""

gpt_response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "Du bist ein Experte für Textzusammenfassungen."},
              {"role": "user", "content": gpt_prompt}],
    temperature=0.2
)

# Antwort extrahieren
summary_result = gpt_response.choices[0].message.content

# Markdown-Formatierte Ausgabe
markdown_output = f"""
## 🎙️ Transkript der Audiodatei

**📂 Datei:** `{audio_file_path}`
**🤖 Modell:** `Whisper-1`

---

{transcribed_text}

---

## ✍️ Zusammenfassung:

{summary_result}
"""

# 📌 Markdown in Google Colab anzeigen
display(Markdown(markdown_output))


## 🎙️ Transkript der Audiodatei

**📂 Datei:** `elevenlabs_erlkoenig_valentino.mp3`
**🤖 Modell:** `Whisper-1`

---

Wer reitet so spät durch Nacht und Wind? Es ist der Vater mit seinem Kind. Er hat den Knaben wohl in dem Arm, Er fasst ihn sicher, er hält ihn warm. Mein Sohn, was birgst du so bang dein Gesicht? Siehst, Vater, du den Erlkönig nicht? Den Erlenkönig mit Kron und Schweif? Mein Sohn, es ist ein Nebelstreif. Du liebes Kind, komm, geh mit mir, Gar schöne Spiele spiel ich mit dir, Manch bunte Blumen sind an dem Strand, Meine Mutter hat manch Güldengewand. Mein Vater, mein Vater, und hörest du nicht, Was Erlenkönig mir leise verspricht? Sei ruhig, bleibe ruhig, mein Kind, In dürren Blättern säuselt der Wind. Willst, feiner Knabe, du mit mir gehen? Meine Töchter sollen dich warten schön, Meine Töchter führen den nächtlichen Reihen Und wiegen und tanzen und singen dich ein. Mein Vater, mein Vater, und siehst du nicht dort Erlkönigs Töchter am düstern Ort? Mein Sohn, mein Sohn, ich seh es genau, Es scheinen die alten Weiden so grau. Ich liebe dich, mich reizt deine schöne Gestalt, Und bist du nicht willig, so brauch ich Gewalt. Mein Vater, mein Vater, jetzt fasst er mich an, Erlkönig hat mir ein Leids getan. Dem Vater grausets, er reitet geschwind, Er hält in Armen das ächzende Kind, Er reicht den Hof mit Mühe und Not, In seinen Armen das Kind war tot.

---

## ✍️ Zusammenfassung:

In dem Gedicht "Erlkönig" von Johann Wolfgang von Goethe reitet ein Vater mit seinem Sohn durch die Nacht, während der Junge von einem geheimnisvollen Erlkönig spricht, der ihn verlockt. Trotz der beruhigenden Worte des Vaters wird der Junge zunehmend ängstlicher, da der Erlkönig ihm droht. Am Ende erreicht der Vater verzweifelt sein Ziel, doch das Kind ist in seinen Armen gestorben. Das Gedicht thematisiert die Angst vor dem Unbekannten und die Ohnmacht der Eltern gegenüber übernatürlichen Kräften.



# 5 | Audio-/Sprachanalyse
---

Durch die Analyse von Inhalten, Tonlage, Betonung und Sprachmelodie kann die emotionale Stimmung in einer Aufnahme erkannt werden. Ein Fokus in diesem Modul ist die Stimmungsanalyse aus Texten, auch **Sentiment Analysis** genannt, sie ist eine Methode zur automatischen Erkennung der emotionalen Haltung in geschriebenen Inhalten. Mithilfe natürlicher Sprachverarbeitung (NLP) werden Texte auf positive, neutrale oder negative Stimmungen untersucht. Diese Technik wird häufig im Kundenfeedback, in sozialen Medien oder in Produktbewertungen eingesetzt, um Meinungen systematisch auszuwerten.  

Moderne KI-Modelle analysieren nicht nur einzelne Wörter, sondern berücksichtigen auch den Kontext, um Stimmungen präziser zu erfassen. Unternehmen nutzen diese Technologie, um Trends zu erkennen, Stimmungen zu messen oder die Kundenzufriedenheit zu verbessern.

In [29]:
import openai
from google.colab import files
from IPython.display import display, Markdown

# 📂 Nutzer lädt eine Audiodatei hoch
# uploaded = files.upload()
# audio_file_path = list(uploaded.keys())[0]

# Die Audiodatei wird direkt verwendet, anstatt sie hochzuladen
audio_file_path = "elevenlabs_erlkoenig_valentino.mp3"

# 🎙️ Audiodatei transkribieren mit Whisper
with open(audio_file_path, "rb") as audio_file:
    response = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )

# 📝 Transkribierter Text
transcribed_text = response.text

# 🧠 Schema für eine Audio/Sprachanalyse
prompt = f"""

Hauptpunkt: Inhaltsanalyse:
    Themen und Argumentationsstruktur: Was wird gesagt? Wie ist das Gesagte aufgebaut?
    Sprachstil: Alltagssprache vs. Fachsprache, formell vs. informell
    Rhetorische Mittel: Metaphern, Wiederholungen, Fragen, Ironie, Pausen
    Satzstruktur & Wortwahl: Einfach oder komplex? Viele Füllwörter?
    Erzählperspektive: Ich-Form, Du-Ansprache, distanziert?
    Tonfall und Haltung: Neutral, ironisch, kritisch, emotional?

Hauptpunkt: Parasprachliche Merkmale (Stimme, Ton, Sprechweise):
    Sprechtempo: Schnell, langsam, variierend
    Intonation: Monoton oder lebendig? Betonung gezielt eingesetzt?
    Lautstärke: Konstant, wechselnd, passend zur Situation?
    Stimmqualität: Klar, nasal, heiser, angespannt?
    Pausen: Natürlich oder unnatürlich? Strategisch gesetzt?
    Versprecher oder Selbstkorrekturen: Häufig? Sympathisch oder irritierend?

Hauptpunkt: Wirkung auf die Zuhörer:innen
    Verständlichkeit: Wird der Inhalt klar transportiert?
    Authentizität: Wirkt die Person glaubwürdig und echt?
    Emotionale Ansprache: Berührt der Inhalt? Motiviert? Regt zum Denken an?
    Zielgruppenorientierung: Passt Sprache und Ton zum Publikum?

Hauptpunkt Stimmungsanalyse:
    Stimmung: Positiv 😊, Neutral 😐, Negativ 😞
    Begründe Deine Einschätzung

Hauptpunkt: Technische Aspekte
    Audioqualität: Rauschfrei, Hall, Hintergrundgeräusche?
    Mikrofonierung: Richtiger Abstand, Pop-Geräusche, Aussteuerung?
    Schnitt & Nachbearbeitung: Erkennbar geschnitten? Übergänge sauber?

Hauptpunkt: Weitere Aspekte:
    Gibt es ethische Aspekte die angesprochen werden müssten.
    Gibt es versteckte Botschaften?
    Welche Punkte könnten noch interessant sein, an die ich nicht gedacht habe?

Vermeide Formulierungen wie könnte oder sollte. Nimm konkret Stellung.

Stelle bei der Ausgabe der Ergebniss nur bei den Hauptpunkte zur besseren Unterscheidung ein "🔹" voran.

### Text:
{transcribed_text}
"""

In [30]:
# Audio-/Sprachanalyse aufrufen
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "Du bist ein Experte für Stimmungsanalysen."},
              {"role": "user", "content": prompt}],
    temperature=0.2
)

# Antwort extrahieren
result = response.choices[0].message.content

In [31]:
# Ausgabe
# 📜 Transkript-Header erstellen
transcript_header = f"""
## 🎙️ Transkript der Audiodatei

**📂 Datei:** `{audio_file_path}`
**🤖 Modell:** `Whisper-1`

---
"""
# Header anzeigen
display(Markdown(transcript_header))

# 📝 Transkriptionstext anzeigen
display(Markdown(transcribed_text))

# 📊 Trennlinie hinzufügen
display(Markdown("---"))

# 🧐 Sprachanalyse anzeigen
header = "## 🧐 Audio-/Sprachanalyse:"
display(Markdown(header))
display(Markdown(result))


## 🎙️ Transkript der Audiodatei

**📂 Datei:** `elevenlabs_erlkoenig_valentino.mp3`
**🤖 Modell:** `Whisper-1`

---


Wer reitet so spät durch Nacht und Wind? Es ist der Vater mit seinem Kind. Er hat den Knaben wohl in dem Arm, Er fasst ihn sicher, er hält ihn warm. Mein Sohn, was birgst du so bang dein Gesicht? Siehst, Vater, du den Erlkönig nicht? Den Erlenkönig mit Kron und Schweif? Mein Sohn, es ist ein Nebelstreif. Du liebes Kind, komm, geh mit mir, Gar schöne Spiele spiel ich mit dir, Manch bunte Blumen sind an dem Strand, Meine Mutter hat manch Güldengewand. Mein Vater, mein Vater, und hörest du nicht, Was Erlenkönig mir leise verspricht? Sei ruhig, bleibe ruhig, mein Kind, In dürren Blättern säuselt der Wind. Willst, feiner Knabe, du mit mir gehen? Meine Töchter sollen dich warten schön, Meine Töchter führen den nächtlichen Reihen Und wiegen und tanzen und singen dich ein. Mein Vater, mein Vater, und siehst du nicht dort Erlkönigs Töchter am düstern Ort? Mein Sohn, mein Sohn, ich seh es genau, Es scheinen die alten Weiden so grau. Ich liebe dich, mich reizt deine schöne Gestalt, Und bist du nicht willig, so brauch ich Gewalt. Mein Vater, mein Vater, jetzt fasst er mich an, Erlkönig hat mir ein Leids getan. Dem Vater grausets, er reitet geschwind, Er hält in Armen das ächzende Kind, Er reicht den Hof mit Mühe und Not, In seinen Armen das Kind war tot.

---

## 🧐 Audio-/Sprachanalyse:

🔹 **Inhaltsanalyse:**
- **Themen und Argumentationsstruktur:** Der Text behandelt die düstere und tragische Begegnung zwischen einem Vater und dem Erlkönig, einem übernatürlichen Wesen. Die Struktur ist dialogisch aufgebaut, wobei der Vater und der Sohn abwechselnd sprechen. Der Vater versucht, seinen Sohn zu beruhigen, während der Sohn die Bedrohung durch den Erlkönig wahrnimmt.
- **Sprachstil:** Der Sprachstil ist poetisch und verwendet eine Mischung aus Alltagssprache und literarischer Sprache. Er ist formell, passend zur düsteren Thematik.
- **Rhetorische Mittel:** Der Text enthält Metaphern (z.B. „Nebelstreif“), Wiederholungen (z.B. „Mein Vater, mein Vater“), Fragen (z.B. „Siehst, Vater, du den Erlkönig nicht?“) und Ironie in der Unterschätzung der Gefahr. Pausen sind implizit durch den Dialog gegeben.
- **Satzstruktur & Wortwahl:** Die Satzstruktur ist überwiegend einfach, mit einigen komplexeren Sätzen. Füllwörter sind minimal, was die Dramatik des Textes verstärkt.
- **Erzählperspektive:** Der Text ist in der dritten Person verfasst, mit direkter Ansprache des Vaters durch den Sohn.
- **Tonfall und Haltung:** Der Tonfall ist emotional und angespannt, mit einer zunehmenden Dramatik, die die Bedrohung durch den Erlkönig verdeutlicht.

🔹 **Parasprachliche Merkmale (Stimme, Ton, Sprechweise):**
- **Sprechtempo:** Das Tempo kann als variierend betrachtet werden, da es zwischen ruhigen und spannungsgeladenen Passagen wechselt.
- **Intonation:** Die Intonation ist lebendig, mit gezielter Betonung auf den emotionalen Höhepunkten.
- **Lautstärke:** Die Lautstärke könnte variieren, um die Dramatik zu unterstreichen, wobei sie in den kritischen Momenten ansteigt.
- **Stimmqualität:** Die Stimmqualität ist klar, was die Verständlichkeit fördert.
- **Pausen:** Pausen sind strategisch gesetzt, um Spannung aufzubauen und die Emotionen zu verstärken.
- **Versprecher oder Selbstkorrekturen:** Diese sind im Text nicht vorhanden, was die Klarheit und den Fluss der Erzählung unterstützt.

🔹 **Wirkung auf die Zuhörer:innen:**
- **Verständlichkeit:** Der Inhalt wird klar transportiert, trotz der poetischen Sprache.
- **Authentizität:** Die Figuren wirken glaubwürdig und echt, was die emotionale Wirkung verstärkt.
- **Emotionale Ansprache:** Der Text berührt stark, regt zum Nachdenken über Verlust und Angst an.
- **Zielgruppenorientierung:** Die Sprache und der Ton sind passend für ein Publikum, das sich mit literarischen Texten und Märchen beschäftigt.

🔹 **Stimmungsanalyse:**
- **Stimmung:** Negativ 😞
- **Begründung:** Die Stimmung ist durch die tragische und bedrohliche Situation geprägt. Der Verlust des Kindes am Ende und die ständige Angst des Sohnes erzeugen eine düstere Atmosphäre.

🔹 **Technische Aspekte:**
- **Audioqualität:** Der Text ist als schriftliche Form gegeben, daher keine Audioqualität zu bewerten.
- **Mikrofonierung:** Entfällt, da es sich um einen schriftlichen Text handelt.
- **Schnitt & Nachbearbeitung:** Auch hier nicht relevant, da es sich um einen literarischen Text handelt.

🔹 **Weitere Aspekte:**
- **Ethische Aspekte:** Der Text behandelt Themen wie Angst, Verlust und die Unfähigkeit der Eltern, ihre Kinder zu schützen, was ethische Überlegungen zur Verantwortung von Eltern aufwirft.
- **Versteckte Botschaften:** Der Text könnte als Warnung vor den Gefahren interpretiert werden, die in der Fantasie verborgen sind, und vor der Unfähigkeit, die Realität von Illusionen zu unterscheiden.
- **Interessante Punkte:** Die symbolische Bedeutung des Erlkönigs als Verkörperung des Todes oder der Bedrohung könnte weiter untersucht werden. Auch die Rolle der Natur und des Übernatürlichen in der Erzählung bietet Raum für tiefere Analysen.

# 6 | Audio-Pipeline
---

Eine Audio-Pipleine kombiniert verschiedene Verarbeitungsschritte, um gesprochene Inhalte effizient zu analysieren, zu verarbeiten und bereitzustellen. Durch die Verknüpfung mehrerer Technologien entstehen leistungsstarke Anwendungen, die Sprache in mehreren Stufen umwandeln und auswerten können.

`Audio → STT → Chat → TTS  `

**Hinweis:**

Die hier vorgestellte Technik wird genutzt, um Audioaufnahmen in Colab zu erstellen und wurde mit ChatGPT und [Link](https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be) erstellt.



<p><font color='black' size="5">
Audio aufzeichnen
</font></p>

Dieser Code erstellt eine einfache Audioaufnahme-Anwendung mit Gradio. Hier ist was er macht:

1. Der Code importiert benötigte Bibliotheken:
   - `gradio` für die Erstellung der Benutzeroberfläche
   - `scipy.io.wavfile` zum Speichern von Audiodateien
   - `numpy` für die Verarbeitung numerischer Daten
   - `files` aus Google Colab zum Herunterladen von Dateien

2. Die Funktion `aufnehmen_und_speichern()` verarbeitet Audiodaten:
   - Sie prüft, ob Audio aufgenommen wurde
   - Bei vorhandenen Audiodaten extrahiert sie die Abtastrate und die Audiodaten
   - Speichert die Aufnahme als "audio.wav"
   - Löst den Download der Datei aus (in Google Colab)
   - Gibt die Audiodaten zurück, damit sie in der Oberfläche abgespielt werden können

3. Eine Gradio-Oberfläche wird erstellt mit:
   - Einem Audio-Input (Mikrofon) für die Aufnahme
   - Einem Audio-Output zum Abspielen der aufgenommenen Datei
   - Einem Titel und einer Beschreibung

4. Schließlich wird die Anwendung mit `demo.launch()` gestartet

Zusammengefasst ist dies eine kompakte Webanwendung, die Audio aufnimmt, es speichert, automatisch zum Download anbietet und gleichzeitig die Möglichkeit bietet, die Aufnahme anzuhören.

In [11]:
# Import
import gradio as gr
import scipy.io.wavfile as wav
import numpy as np
from google.colab import files
from IPython.display import display, HTML, Audio, Markdown
from openai import OpenAI
from pydub import AudioSegment

In [12]:
# Funktion Aufnehmen
def aufnehmen_und_speichern(audio):
    if audio is not None:
        # audio ist ein Tupel (sample_rate, data)
        sample_rate, data = audio

        # Speichern der aufgenommenen Datei
        wav.write("audio_in.wav", sample_rate, data)

        # Download auslösen
        files.download("audio_in.wav")

        # Wichtig: Die aufgenommene Audiodatei korrekt zurückgeben
        return (sample_rate, data)
    else:
        return None

In [13]:
demo = gr.Interface(
    fn=aufnehmen_und_speichern,
    inputs=gr.Audio(sources="microphone", type="numpy"),
    outputs=gr.Audio(type="numpy"),  # Setze den gleichen Typ für Output
    title="🎙️ Audio Aufnahme",
    description="Nimm ein Audio auf und und speichert es.",
    flagging_mode="never"
)

In [14]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://05594d9eb55837d7b1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<p><font color='black' size="5">
STT
</font></p>

Dieser Code verwendet OpenAI's Whisper-Modell, um eine automatische Transkription einer Audiodatei durchzuführen.

In [16]:
client = OpenAI()

def transcribe_audio(filename):
    with open(filename, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    return transcription.text

# Transkribieren Sie die aufgezeichneten Audiodaten
transcription = transcribe_audio("/content/audio_in.wav")

display(Markdown("## 📄 Transkribiert:"))
display(Markdown(f"---"))
display(Markdown(f"{transcription}"))

## 📄 Transkribiert:

---

Drei Gründe, warum ich generative KI in meinen Job einsetzen sollte.

<p><font color='black' size="5">
Chat
</font></p>

In [17]:
def query_llm(prompt):
  response = client.chat.completions.create(
      model="gpt-4o-mini",  # Use the specified model
      messages=[{"role": "user", "content": prompt}],
      temperature=0.2  # Adjust temperature as needed
  )
  return response.choices[0].message.content

#
user_prompt = transcription
llm_response = query_llm(user_prompt)

display(Markdown("## 🤖 Chat"))
display(Markdown(f"---"))
display(Markdown(f"**Prompt:** {user_prompt}"))
display(Markdown(f"**LLM Antwort:** {llm_response}"))

## 🤖 Chat

---

**Prompt:** Drei Gründe, warum ich generative KI in meinen Job einsetzen sollte.

**LLM Antwort:** Hier sind drei Gründe, warum der Einsatz von generativer KI in deinem Job vorteilhaft sein könnte:

1. **Effizienzsteigerung**: Generative KI kann repetitive Aufgaben automatisieren und somit Zeit sparen. Sie kann beispielsweise bei der Erstellung von Berichten, der Analyse von Daten oder der Generierung von Inhalten helfen. Dadurch kannst du dich auf strategischere und kreativere Aspekte deiner Arbeit konzentrieren.

2. **Kreative Unterstützung**: Generative KI kann als kreatives Werkzeug dienen, indem sie neue Ideen, Konzepte oder Designs vorschlägt. Sie kann dir helfen, Inspiration zu finden und innovative Lösungen für Probleme zu entwickeln, die du möglicherweise nicht alleine in Betracht gezogen hättest.

3. **Personalisierung und Anpassung**: Mit generativer KI kannst du Inhalte oder Dienstleistungen besser auf die Bedürfnisse deiner Zielgruppe zuschneiden. Sie kann Daten analysieren und personalisierte Empfehlungen oder Lösungen anbieten, was zu einer höheren Kundenzufriedenheit und -bindung führen kann.

Insgesamt kann der Einsatz von generativer KI dazu beitragen, deine Arbeit effizienter, kreativer und kundenorientierter zu gestalten.

<p><font color='black' size="5">
TTS
</font></p>

Der Code nimmt einen Text als Eingabe, wandelt ihn mithilfe eines KI-gestützten Text-to-Speech-Modells (`tts-1`) in gesprochene Sprache um und gibt die generierte Audiodatei aus.

In [18]:
def generate_text(text, voice="onyx"):
    response = client.audio.speech.create(
        model="tts-1",
        voice=voice,
        input=text
    )
    audio_data = response.content
    return audio_data  # Audiodaten direkt zurückgeben

def speak_text(text):
    display(Markdown("## 🎤 ... Audio wird jetzt erstellt ..."))
    audio_data = generate_text(text)
    with open("audio_out.mp3", "wb") as audio_file:
        audio_file.write(audio_data)
    print()
    display(Markdown("## 🔊 ... Speech"))
    display(Markdown("---"))
    print()
    display(Audio(audio_data))

display(Markdown("## 📄 Text to ..."))
display(Markdown(f"---"))
display(Markdown(f"{llm_response}"))
display(Markdown(f"---"))
speak_text(llm_response)

## 📄 Text to ...

---

Hier sind drei Gründe, warum der Einsatz von generativer KI in deinem Job vorteilhaft sein könnte:

1. **Effizienzsteigerung**: Generative KI kann repetitive Aufgaben automatisieren und somit Zeit sparen. Sie kann beispielsweise bei der Erstellung von Berichten, der Analyse von Daten oder der Generierung von Inhalten helfen. Dadurch kannst du dich auf strategischere und kreativere Aspekte deiner Arbeit konzentrieren.

2. **Kreative Unterstützung**: Generative KI kann als kreatives Werkzeug dienen, indem sie neue Ideen, Konzepte oder Designs vorschlägt. Sie kann dir helfen, Inspiration zu finden und innovative Lösungen für Probleme zu entwickeln, die du möglicherweise nicht alleine in Betracht gezogen hättest.

3. **Personalisierung und Anpassung**: Mit generativer KI kannst du Inhalte oder Dienstleistungen besser auf die Bedürfnisse deiner Zielgruppe zuschneiden. Sie kann Daten analysieren und personalisierte Empfehlungen oder Lösungen anbieten, was zu einer höheren Kundenzufriedenheit und -bindung führen kann.

Insgesamt kann der Einsatz von generativer KI dazu beitragen, deine Arbeit effizienter, kreativer und kundenorientierter zu gestalten.

---

## 🎤 ... Audio wird jetzt erstellt ...

## 🔊 ... Speech

---

<p><font color='black' size="5">
Audio-Podcast
</font></p>


Ein Podcast ist eine Serie von Audio- oder Videoepisoden, die über das Internet bereitgestellt und meist abonniert werden können. Podcasts sind besonders beliebt, weil sie flexibel konsumiert werden können – ob unterwegs, beim Sport oder zu Hause. Die Themenvielfalt reicht von Nachrichten, Bildung und Wissenschaft bis hin zu Unterhaltung, Interviews und persönlichen Geschichten. Um einen Podcast zu hören, benötigt man lediglich eine Podcast-App oder eine entsprechende Webseite.

**Podcast zum Kurns GenAI:**

[DeepDive GenAI](https://open.spotify.com/show/4nxwhKtK2tvNeDoHS91iY9?si=qLmrGhxmStu6NpZD6_DLZg&nd=1&dlsi=b1bfd4c659384aad)

**WebApps:**

[NotebookLM](https://notebooklm.google.com/)     
[jellypod](https://jellypod.ai/)

Einleitung generieren ...

In [19]:
import random
import time

# Setze einen zufälligen Seed basierend auf der aktuellen Zeit
random.seed(time.time() * 1000000)

# Zufälliges Vorwort
vorwort = [
"""
Schön, dass du dabei bist! In diesem Podcast sprechen wir über Themen, die uns wirklich beschäftigen. Heute nehmen wir ein Thema unter die Lupe, das Dich auch schon mal interessiert hat.
""",
"""
Hallo und herzlich willkommen! Heute geht’s um ein Thema, das aktueller nicht sein könnte – und das dich direkt betrifft.
""",
"""
Willkommen beim Podcast über die großen und kleinen Fragen unseres Alltags. Heute geht’s um ein Thema, das für Gesprächsstoff sorgt – bleib dran!
"""
]

index = random.randint(0, len(vorwort) - 1)
intro = vorwort[index]

In [20]:
display(Markdown("## 🎤 ... Prolog wird erstellt ..."))
audio_data = generate_text(intro, "nova")

with open("audio_prolog.mp3", "wb") as audio_file:
    audio_file.write(audio_data)

## 🎤 ... Prolog wird erstellt ...

Frage-Audio mit neuer Stimme generieren ...

In [21]:
display(Markdown("## 🎤 ... Frage wird mit neuer Stimme erstellt ..."))
audio_data = generate_text(transcription, "nova")
with open("audio_in.mp3", "wb") as audio_file:
    audio_file.write(audio_data)

## 🎤 ... Frage wird mit neuer Stimme erstellt ...

Nachwort generieren ...

In [22]:
# Setze einen zufälligen Seed basierend auf der aktuellen Zeit
random.seed(time.time() * 1000000)

# Zufälliges Nachwort
nachwort = [
"""
Puh, ganz schön viel Stoff heute! Nimm dir Zeit, das alles wirken zu lassen – und wenn’s dir gefallen hat, abonnier gern den Podcast. Bis zum nächsten Mal!
""",
"""
Krass! Das waren viele spannende Einblicke – schön, dass du dabei warst! Lass das Gehörte in Ruhe auf dich wirken, und wir freuen uns, wenn du beim nächsten Mal wieder reinhörst. Bis bald!
""",
"""
Wow! Danke, dass du heute dabei warst. Wenn du Anregungen oder Fragen hast, schreib uns gern. Wir freuen uns schon auf die nächste Folge mit dir – bis bald!
"""
]

index = random.randint(0, len(nachwort) - 1)
outtro = nachwort[index]

In [23]:
display(Markdown("## 🎤 ... Nachwort wird erstellt ..."))
audio_data = generate_text(outtro, "nova")

with open("audio_epilog.mp3", "wb") as audio_file:
    audio_file.write(audio_data)

## 🎤 ... Nachwort wird erstellt ...

Audios werden kombiniert

In [24]:
# Audiodateien laden
audio1 = AudioSegment.from_file("audio_prolog.mp3")
audio2 = AudioSegment.from_file("audio_in.mp3")
audio3 = AudioSegment.from_file("audio_out.mp3")
audio4 = AudioSegment.from_file("audio_epilog.mp3")
display(Markdown("## 🎤 ... Audios werden kombiniert ..."))

# Sekunden Stille erstellen (Dauer in Millisekunden)
pause = AudioSegment.silent(duration=1000)  # 1000 ms = 1 Sekunden

# Audiodateien verketten
audio_combi = audio1 + audio2 + pause + audio3 + pause + audio4

# Ergebnis speichern
audio_combi.export("audio_combi.mp3", format="mp3")

## 🎤 ... Audios werden kombiniert ...

<_io.BufferedRandom name='audio_combi.mp3'>

Audio ausgeben

In [25]:
# Ausgabe
print()
display(Markdown("## 🔊 ... Speech"))
display(Markdown("---"))
print()

# Anzeigen der gespeicherten Datei statt des AudioSegment-Objekts
display(Audio("audio_combi.mp3", autoplay=False))

## 🔊 ... Speech

---

# A | Aufgabe
---



Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
Mehrsprachiger Audioübersetzer
</font></p>




**Aufgabenstellung:**     
Entwickln Sie einen Audioübersetzer, der:

+ Eine deutsche Audiodatei transkribiert
+ Den transkribierten Text in eine andere Sprache deiner Wahl übersetzt (z.B. Englisch, Spanisch, Französisch)
+ Die Übersetzung in eine Audiodatei mit passender Stimme umwandelt
+ Eine Benutzeroberfläche bietet, die den Originaltext, die Übersetzung und beide Audiodateien anzeigt

**Hinweis:**

Verwende Whisper für die Transkription, aber die Übersetzung muss mit einem separaten Prompt an das LLM erfolgen. Wähle eine Stimme, die zur Zielsprache passt. Die Lösung soll keine direkte Kopie der Beispiele sein, sondern eine eigene Integration der Komponenten.
Lösen Sie die Aufgaben Schritt für Schritt, dass zumindest Teillösungen vorliegen.


<p><font color='black' size="5">
Audio-Inhaltsklassifikator
</font></p>


**Aufgabenstellung:**   

Erstellen Sie ein einfaches Programm, das eine Audiodatei transkribiert und anschließend den Inhalt klassifiziert.

Das Programm soll:

+ Eine Audiodatei mit Whisper transkribieren
+ Den transkribierten Text mit einem LLM analysieren, um festzustellen, ob es sich um:

    + ein Interview
    + einen Monolog/Vortrag
    + ein Gespräch zwischen mehreren Personen
    + eine andere Kategorie handelt

Zusätzlich die Hauptthemen oder Schlüsselwörter aus dem Inhalt identifizieren
Die Ergebnisse übersichtlich ausgeben

**Hinweis:**

Lösen Sie die Aufgaben Schritt für Schritt, dass zumindest Teillösungen vorliegen.